### Preprocessing employed to obtain data

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
from spotipy.oauth2 import SpotifyOAuth
import requests
import pandas as pd
import numpy as np


client_id2 = "a7d5692bf0f744fdb9abf64e9d7d82ab"
client_secret2 = "852229767e7e4e29989d91990383d928"
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id2, client_secret=client_secret2))

# Function to get track name and artist name from SPOTIFY API
def get_track_info(track_ids):
    try:
        tracks = sp.tracks(track_ids)['tracks']
        time.sleep(60/150)
        results = []
        for track in tracks:
            if "error" in track:
                results.append([track_ids[tracks.index(track)], None, None, False])
            else:
                track_name = track['name']
                artist_name = track['artists'][0]['name']
                pop = track["popularity"]
                results.append([track['id'], track_name, artist_name, True, pop])
        return results
    except spotipy.exceptions.SpotifyException as e:
        print(f"SpotifyException: {e.http_status}")
        return [[track_id, None, None, False] for track_id in track_ids]
    
# Function to get artist's genres from SPOTIFY API
def get_artist_genres(artist_name):
    artist_info = sp.search(q=f'artist:{artist_name}', type='artist')
    time.sleep(0.35)
    if artist_info['artists']['items']:
        artist = artist_info['artists']['items'][0]
        genres = artist.get('genres', [])
        return genres
    return []


In [ ]:
df_2021 = pd.read_csv('data_formatted.csv')
df_2021 = df_2021.sort_values(by="popularity", ascending=False)
df_2021

### GET SPOTIFY ARTISTS' GENRES
This code guarantees accessing Spotify's API to obtain the genres on artist level for each of the artists in the dataset.

In [ ]:



df_aux_sp_genres = df_2021.copy()
df_aux_sp_genres['gen_artist'] = None
# Set para almacenar los artistas ya consultados
consulted_artists = set()

# Iterar sobre cada fila del DataFrame
for index, row in df_aux_sp_genres.iterrows():
    artists = str(row['artist_name'])
    print(index)
    if artists != "nan":
        artists = artists.split(', ')
        genres = []
        for artist in artists:
            if artist not in consulted_artists:
                try:
                    artist_genres = get_artist_genres(artist)
                    consulted_artists.add(artist)
                except requests.exceptions.ReadTimeout:
                    print(f"ReadTimeout occurred for artist: {artist}. Waiting for 10 seconds before retrying.")
                    time.sleep(10)
                    artist_genres = get_artist_genres(artist)
            else:
                artist_genres = []
            
            genres.extend(artist_genres)
        
        df_aux_sp_genres.at[index, 'gen_artist'] = list(set(genres))  # Eliminar duplicados
    
    # partial save every 10000 iterations
    # to avoid losing progress in case of an error
    if index % 10000 == 0 and index != 0:
        df_aux_sp_genres.to_csv(f"spotify_artists_genres_2024_partial_{index//10000}.csv", index=False)

df_aux_sp_genres.to_csv("spotify_artists_genres_2024_n.csv", index=False)

df_aux_sp_genres



This code is for obtaining popularity for 2025 for each song in the dataset. It creates 50 songs' batches and then checks its popularity, track_name and artist_name. It only requieres an Spotify track_id to check the API.

In [ ]:

# Initialize an empty list to store track_ids
track_ids = []

# Initialize an empty dataframe to store the results
df_aux = pd.DataFrame(columns=['id', 'track_name', 'artist_name', 'estado_solicitud', "new_popularity_2025"])

# Iterate over the rows of the dataframe prueba
for index, row in df.iterrows():
    print(index)
    track_ids.append(row['id'])
    # When the length of the list is 50, call get_track_info
    if len(track_ids) == 50:
        track_info = get_track_info(track_ids)
        
        # Create a temporary dataframe from the track_info
        temp_df = pd.DataFrame(track_info, columns=['id', 'track_name', 'artist_name', 'estado_solicitud',"new_popularity_2025"])
        
        # Append the temporary dataframe to df_aux
        df_aux = pd.concat([df_aux, temp_df], ignore_index=True)
        
        # Clear the track_ids list
        track_ids = []

# If there are any remaining track_ids, call get_track_info for the last batch
if track_ids:
    track_info = get_track_info(track_ids)
    temp_df = pd.DataFrame(track_info, columns=['id', 'track_name', 'artist_name', 'estado_solicitud',"new_popularity_2025"])
    df_aux = pd.concat([df_aux, temp_df], ignore_index=True)



df_final = pd.merge(df_aux, df, on='id', how='inner')
df_final = df_final.drop(columns=['estado_solicitud'])
df_final